In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\projects\\end-to-end-SMS-Spam-classifier\\research'

In [2]:
import os

os.chdir('../')
%pwd

'c:\\Users\\44787\\Desktop\\projects\\end-to-end-SMS-Spam-classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainingConfig:
    root_dir: Path
    train_path: Path

In [4]:
from src.SMSClassifier.constants import CONFIG_PATH
from src.SMSClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfiguratoinManager:
    def __init__(self, config=CONFIG_PATH):
        self.config = read_yaml(config)

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            train_path=config.train_path,
        )
        return model_training_config

In [6]:
from src.SMSClassifier.logging import logger
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
from src.SMSClassifier.utils.common import save_bin
import joblib
import pandas as pd

In [7]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig) -> None:
        self.config = config

    def training_pipeline(self):

        train_path = pd.read_csv(self.config.train_path)

        X_train = train_path.drop('target', axis=1)
        y_train = train_path['target']

        # train the model
        model = MultinomialNB()
        model.fit(X_train, y_train)

        logger.info(f"Model train score: {model.score(X_train, y_train)}")

        # save the model
        joblib.dump(model, os.path.join(self.config.root_dir, 'model.joblib'))

In [9]:
try:
    config = ConfiguratoinManager()
    model_training_config = config.get_model_training_config()
    model = ModelTraining(model_training_config)
    model.training_pipeline()
except Exception as e:
    raise e

[2024-02-06 16:11:23,193: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-06 16:11:23,196: INFO: common: created directory at: artifacts/model_training]
[2024-02-06 16:11:24,399: INFO: 696275271: Model train score: 0.9760061919504643]
